<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [4]:
import numpy as np
import gensim
import os
import re
import spacy

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd
import string

In [14]:
path = '../module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv'
df = pd.read_csv(path)

In [18]:
df['primaryCategories'].value_counts()

Electronics                    13995
Health & Beauty                12071
Toys & Games,Electronics        1676
Office Supplies,Electronics      386
Electronics,Media                185
Office Supplies                    9
Animals & Pet Supplies             6
Home & Garden                      2
Electronics,Furniture              2
Name: primaryCategories, dtype: int64

In [19]:
df.columns

Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'reviews.date', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs'],
      dtype='object')

In [40]:
reviews = pd.DataFrame(data = df, columns=['primaryCategories', 'reviews.text', 'reviews.rating'])

In [41]:
reviews['reviews.text'] = [re.sub('[^a-zA-Z 0-9]', '', x) for x in reviews['reviews.text']]

In [42]:
reviews.head()

,primaryCategories,reviews.text,reviews.rating
0,Health & Beauty,I order 3 of them and one of the item is bad q...,3
1,Health & Beauty,Bulk is always the less expensive way to go fo...,4
2,Health & Beauty,Well they are not Duracell but for the price i...,5
3,Health & Beauty,Seem to work as well as name brand batteries a...,5
4,Health & Beauty,These batteries are very long lasting the pric...,5


In [43]:
nlp = spacy.load('en_core_web_lg')

def tokenize(text):
    
    doc = nlp(text)
    
    STOP_WORDS = nlp.Defaults.stop_words

    return [token.lemma_.strip().lower().translate(str.maketrans('', '', string.punctuation)) for token in doc
            if (token not in STOP_WORDS)
            and (token not in STOPWORDS)
            and (token.is_punct != True) 
            and (token.like_num != True)
            and (token.is_currency != True)
            and (token.is_digit != True)
            and (token.is_space != True)]

In [45]:
def yield_tokens(docs):
    for doc in docs:
        tokens = tokenize(doc)
    yield tokens

In [49]:
token_stream = yield_tokens(reviews['reviews.text'])

In [ ]:
id2word = corpora.Dictionary(yield_tokens(reviews['reviews.text']))

In [ ]:
id2word.filter_extremes(no_below=5, no_above=0.95)

In [ ]:
corpus = [id2word.doc2bow(text) for text in yield_tokens(reviews['reviews.text'])]

In [ ]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=33,
                   num_topics = 5,
                   passes=10,
                   workers=4
                  )

In [ ]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:5]) for t in words]

In [ ]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling